In [55]:
import numpy as np 

# suppress scientific notation
np.set_printoptions(suppress=True)

# generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size=20)

# make sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# round each number to the second decimal place
params = np.round(params, 2)

# show the paramters
print(params)

[140.5  -35.47   0.     8.92 120.21  46.51 -28.55 103.86  20.8   76.85
 -32.4   32.71  90.   -34.47 -10.2   64.31 139.5   -7.02  63.14  20.21]


In [56]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits-1) -1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale 

def quantization_error(params: np.array, params_q: np.array):
    return np.mean((params - params_q)**2)

(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print("Original: ")
print(np.round(params, 2))
print("")
print(f"Asymetric scale: {asymmetric_scale} | Zero: {asymmetric_zero}")
print(asymmetric_q)
print("")
print(f"Symetric scale: {symmetric_scale}")
print(symmetric_q)

Original: 
[140.5  -35.47   0.     8.92 120.21  46.51 -28.55 103.86  20.8   76.85
 -32.4   32.71  90.   -34.47 -10.2   64.31 139.5   -7.02  63.14  20.21]

Asymetric scale: 0.690078431372549 | Zero: 51.0
[255   0  51  64 225 118  10 202  81 162   4  98 181   1  36 144 253  41
 142  80]

Symetric scale: 1.1062992125984252
[127 -32   0   8 109  42 -26  94  19  69 -29  30  81 -31  -9  58 126  -6
  57  18]


In [57]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[140.5  -35.47   0.     8.92 120.21  46.51 -28.55 103.86  20.8   76.85
 -32.4   32.71  90.   -34.47 -10.2   64.31 139.5   -7.02  63.14  20.21]

Dequantize Asymmetric:
[140.78 -35.19   0.     8.97 120.07  46.24 -28.29 104.2   20.7   76.6
 -32.43  32.43  89.71 -34.5  -10.35  64.18 139.4   -6.9   62.8   20.01]

Dequantize Symmetric:
[140.5  -35.4    0.     8.85 120.59  46.46 -28.76 103.99  21.02  76.33
 -32.08  33.19  89.61 -34.3   -9.96  64.17 139.39  -6.64  63.06  19.91]


In [58]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.04
   Symmetric error: 0.07
